In [1]:
import SimpleITK as sitk
import numpy as np
import pandas as pd
import monai
import os

In [16]:
ct_path = '/home/ikboljon.sobirov/data/nas/ikboljon.sobirov/manor_phantom_nrrd/B_ST00001/'
seg_path = '/home/ikboljon.sobirov/data/nas/ikboljon.sobirov/New folder/Manor phantom B/ST00001/Segmentation.seg.nrrd'
xls_path = '/home/ikboljon.sobirov/data/nas/ikboljon.sobirov/manor_phantom_nrrd/xls_files/Manor B/'

In [17]:
# save_id = ct_path.split('/')[-1]
# # set_id = ct_path.split('/')[-2]
# # save_id = ct_path.split('/')[-3] + '_' + ct_path.split('/')[-2]
# save_id
patients = os.listdir(ct_path)
patients

['80KV 50mA',
 '135KV 50mA',
 '120KV 400mA',
 '100KV 50mA',
 '100KV 600mA',
 '80KV 600mA',
 '80KV 400mA',
 '120KV 600mA',
 '100KV 200mA',
 '135KV 200mA',
 '135KV 400mA',
 '120KV 50mA',
 '120KV 200mA',
 '100KV 400mA',
 '80KV 200mA']

In [18]:
# patients = os.listdir(ct_path)
# patients = [i for i in patients if not i.endswith('AUTOMA')]
# patients.sort()
# patients

In [19]:
for p in patients:
    p_path = os.path.join(ct_path, p, (p+'.nrrd'))
    ct_image = sitk.ReadImage(p_path)
    segment_image = sitk.ReadImage(seg_path)

    resampler = sitk.ResampleImageFilter()
    resampler.SetReferenceImage(ct_image)
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    segment_image = resampler.Execute(segment_image)
    print(ct_image.GetSize(), segment_image.GetSize())
    # check is ct's 3rd dim is larger than 300
    if ct_image.GetSize()[2] < 300: pass

    # Convert to NumPy arrays
    ct_array = sitk.GetArrayFromImage(ct_image)
    segment_array = sitk.GetArrayFromImage(segment_image)

    # Initialize a dictionary to store segment statistics
    segment_statistics = {}

    # Calculate statistics for each segment
    for segment_label in np.unique(segment_array):
        if segment_label == 0:  # Skip background
            continue

        # Create a binary mask for the current segment
        segment_mask = (segment_array == segment_label)

        # Calculate the number of voxels in the segment
        num_voxels = np.sum(segment_mask)

        # Calculate the volume in different units
        voxel_volume = np.prod(ct_image.GetSpacing())
        volume_mm3 = num_voxels * voxel_volume
        volume_cm3 = volume_mm3 / 1000

        # Calculate HU statistics for the segment
        segment_values = ct_array[segment_mask]
        min_hu = np.min(segment_values)
        max_hu = np.max(segment_values)
        mean_hu = np.mean(segment_values)
        median_hu = np.median(segment_values)
        std_hu = np.std(segment_values)

        # Store the statistics in the dictionary
        segment_statistics[segment_label] = {
            "Number of Voxels": num_voxels,
            "Volume (mm³)": volume_mm3,
            "Volume (cm³)": volume_cm3,
            "Minimum HU": min_hu,
            "Maximum HU": max_hu,
            "Mean HU": mean_hu,
            "Median HU": median_hu,
            "Standard Deviation of HU": std_hu
        }


    
    # You can access the statistics for a specific segment using its label
    # segment_label_to_lookup = 1  # Replace with the desired segment label
    # if segment_label_to_lookup in segment_statistics:
    #     statistics_for_segment = segment_statistics[segment_label_to_lookup]
    #     print(f"Statistics for Segment {segment_label_to_lookup}:")
    #     for key, value in statistics_for_segment.items():
    #         print(f"{key}: {value}")
    
    df = pd.DataFrame(segment_statistics)
    df_transposed = df.transpose()

    # if not os.path.exists(os.path.join(xls_path, p)):
    #     os.makedirs(os.path.join(xls_path, p))
        
    # save to xlsx file
    df_transposed.to_excel(os.path.join(xls_path, p+'.xlsx'))

(512, 512, 342) (512, 512, 342)
(512, 512, 346) (512, 512, 346)
(512, 512, 342) (512, 512, 342)
(512, 512, 342) (512, 512, 342)
(512, 512, 346) (512, 512, 346)
(512, 512, 357) (512, 512, 357)
(512, 512, 349) (512, 512, 349)
(512, 512, 349) (512, 512, 349)
(512, 512, 342) (512, 512, 342)
(512, 512, 342) (512, 512, 342)
(512, 512, 346) (512, 512, 346)
(512, 512, 361) (512, 512, 361)
(512, 512, 353) (512, 512, 353)
(512, 512, 361) (512, 512, 361)
(512, 512, 349) (512, 512, 349)


In [15]:
df_transposed

,Number of Voxels,Volume (mm³),Volume (cm³),Minimum HU,Maximum HU,Mean HU,Median HU,Standard Deviation of HU
1,9355.0,515.572760,0.515573,-137.0,-44.0,-91.647354,-92.0,12.850136
2,9355.0,515.572760,0.515573,0.0,96.0,44.873437,45.0,13.836003
3,9355.0,515.572760,0.515573,62.0,168.0,115.524319,116.0,12.703851
4,9355.0,515.572760,0.515573,-2.0,98.0,53.777018,54.0,13.117684
5,9355.0,515.572760,0.515573,-16.0,81.0,34.275468,34.0,12.795665
6,9355.0,515.572760,0.515573,-226.0,-120.0,-182.482309,-182.0,14.097582
7,9355.0,515.572760,0.515573,-176.0,-90.0,-135.692250,-136.0,12.319812
8,9355.0,515.572760,0.515573,452.0,561.0,505.243827,505.0,16.141196
9,9355.0,515.572760,0.515573,-135.0,-49.0,-95.570283,-96.0,11.842718
10,9355.0,515.572760,0.515573,-4.0,82.0,40.211545,40.0,11.744848


In [10]:


# # Load the CT data and mask segments
# ct_image = sitk.ReadImage(ct_path)
# segment_image = sitk.ReadImage(seg_path)

# resampler = sitk.ResampleImageFilter()
# resampler.SetReferenceImage(ct_image)
# resampler.SetInterpolator(sitk.sitkNearestNeighbor)
# segment_image = resampler.Execute(segment_image)

# print(ct_image.GetSize(), segment_image.GetSize())


# # Convert to NumPy arrays
# ct_array = sitk.GetArrayFromImage(ct_image)
# segment_array = sitk.GetArrayFromImage(segment_image)

# # Initialize a dictionary to store segment statistics
# segment_statistics = {}

# # Calculate statistics for each segment
# for segment_label in np.unique(segment_array):
#     if segment_label == 0:  # Skip background
#         continue

#     # Create a binary mask for the current segment
#     segment_mask = (segment_array == segment_label)

#     # Calculate the number of voxels in the segment
#     num_voxels = np.sum(segment_mask)

#     # Calculate the volume in different units
#     voxel_volume = np.prod(ct_image.GetSpacing())
#     volume_mm3 = num_voxels * voxel_volume
#     volume_cm3 = volume_mm3 / 1000

#     # Calculate HU statistics for the segment
#     segment_values = ct_array[segment_mask]
#     min_hu = np.min(segment_values)
#     max_hu = np.max(segment_values)
#     mean_hu = np.mean(segment_values)
#     median_hu = np.median(segment_values)
#     std_hu = np.std(segment_values)

#     # Store the statistics in the dictionary
#     segment_statistics[segment_label] = {
#         "Number of Voxels": num_voxels,
#         "Volume (mm³)": volume_mm3,
#         "Volume (cm³)": volume_cm3,
#         "Minimum HU": min_hu,
#         "Maximum HU": max_hu,
#         "Mean HU": mean_hu,
#         "Median HU": median_hu,
#         "Standard Deviation of HU": std_hu
#     }

# # You can access the statistics for a specific segment using its label
# segment_label_to_lookup = 1  # Replace with the desired segment label
# if segment_label_to_lookup in segment_statistics:
#     statistics_for_segment = segment_statistics[segment_label_to_lookup]
#     print(f"Statistics for Segment {segment_label_to_lookup}:")
#     for key, value in statistics_for_segment.items():
#         print(f"{key}: {value}")


(512, 512, 509) (512, 512, 509)
Statistics for Segment 1:
Number of Voxels: 5395
Volume (mm³): 514.5072937011719
Volume (cm³): 0.5145072937011719
Minimum HU: 35
Maximum HU: 121
Mean HU: 71.78609823911029
Median HU: 72.0
Standard Deviation of HU: 10.878494462630462


In [11]:
df = pd.DataFrame(segment_statistics)

In [12]:
df_transposed = df.transpose()
df_transposed

,Number of Voxels,Volume (mm³),Volume (cm³),Minimum HU,Maximum HU,Mean HU,Median HU,Standard Deviation of HU
1,5395.0,514.507294,0.514507,35.0,121.0,71.786098,72.0,10.878494
2,5395.0,514.507294,0.514507,128.0,207.0,165.696571,166.0,11.093388
3,5395.0,514.507294,0.514507,-48.0,42.0,-0.914736,-1.0,11.336154
4,5395.0,514.507294,0.514507,456.0,565.0,509.070250,509.0,15.614544
5,5395.0,514.507294,0.514507,-136.0,-52.0,-94.161446,-94.0,11.932591
6,5395.0,514.507294,0.514507,5.0,92.0,40.741613,41.0,10.436496
7,5710.0,544.548035,0.544548,-799.0,-727.0,-759.461471,-759.0,9.677524
8,5395.0,514.507294,0.514507,284.0,388.0,336.425394,337.0,13.839117
9,5395.0,514.507294,0.514507,34.0,109.0,73.023355,73.0,10.693463
10,5395.0,514.507294,0.514507,133.0,203.0,166.960890,167.0,10.783442


In [13]:
import monai

# Define input and output file paths
dicom_folder_path = '/Users/ikboljon.sobirov/Documents/nsclc/manifest-1603198545583/NSCLC-Radiomics'
nrrd_file_path = '/Users/ikboljon.sobirov/Downloads/Phantom example/file.nrrd'


In [14]:
!pip install pydicom pynrrd


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 1.7 MB/s eta 0:00:00a 0:00:010m


In [15]:
import os
import pydicom
import nrrd

def dicom_to_nrrd(dicom_dir, nrrd_file):
    # Create an empty list to store DICOM slices
    dicom_slices = []

    # Loop through DICOM files in the specified directory
    for root, _, files in os.walk(dicom_dir):
        for file in files:
            if file.lower().endswith(".dcm"):
                dicom_file = os.path.join(root, file)
                dicom_slices.append(pydicom.dcmread(dicom_file))

    if not dicom_slices:
        print("No DICOM files found in the specified directory.")
        return

    # Sort the DICOM slices based on their ImagePositionPatient
    dicom_slices.sort(key=lambda x: x.ImagePositionPatient[2])

    # Extract pixel data from DICOM slices
    pixel_data = [ds.pixel_array for ds in dicom_slices]

    # Get the voxel spacing
    spacing = [
        float(dicom_slices[0].PixelSpacing[0]),
        float(dicom_slices[0].PixelSpacing[1]),
        float(dicom_slices[0].SliceThickness)
    ]

    # Convert pixel data to NRRD format
    nrrd.write(nrrd_file, pixel_data, header={"spacings": spacing})

    print(f"Conversion complete. NRRD file saved as {nrrd_file}.")


In [19]:
dicom_folder_path = '/Users/ikboljon.sobirov/Documents/nsclc/manifest-1603198545583/NSCLC-Radiomics'
nrrd_file_path = '/Users/ikboljon.sobirov/Downloads/Phantom example/file.nrrd'

# # Example usage:
# dicom_directory = "path/to/dicom_folder"
# nrrd_output_file = "output.nrrd"
# dicom_to_nrrd(dicom_folder_path, nrrd_file_path)


In [20]:
import SimpleITK as sitk
import os

def dicom_to_nrrd(dicom_dir, nrrd_file):
    # Create a reader for DICOM files
    reader = sitk.ImageSeriesReader()

    dicom_files = reader.GetGDCMSeriesFileNames(dicom_dir)
    reader.SetFileNames(dicom_files)

    # Load the DICOM series
    dicom_image = reader.Execute()

    # Save the DICOM series as NRRD
    sitk.WriteImage(dicom_image, nrrd_file)

    print(f"Conversion complete. NRRD file saved as {nrrd_file}.")


In [21]:
nrrd_file_path2 = '/Users/ikboljon.sobirov/Downloads/Phantom example/file2.nrrd'

dicom_to_nrrd(dicom_folder_path, nrrd_file_path2)


RuntimeError: Exception thrown in SimpleITK ImageSeriesReader_Execute: /tmp/SimpleITK/Code/IO/src/sitkImageSeriesReader.cxx:129:
sitk::ERROR: File names information is empty. Cannot read series.